# Import and setup

In [1]:
import pandas as pd
import json
from copy import deepcopy
import numpy as np
import math
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
getDashboard = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/Du lieu api/getDashboard.csv")
getDashboard["CYCLETIME"] = getDashboard["CYCLETIME"].apply(lambda x: "THANG" if x == 2 else "NGAY")
getListChart = pd.read_excel("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/Du lieu api/getListChart_202309121406.xlsx")

In [3]:
getDashboard.head()

,GROUPPROFILECODE,CYCLETIME,AREACODE,DASHBOARDID,DASHBOARDNAME
0,VPTD,NGAY,H031,3581,Doanh thu dịch vụ nhóm VTT
1,VPTD,NGAY,H031,3582,Doanh thu dịch vụ nhóm VDS
2,VPTD,THANG,H321,3517,Doanh thu dịch vụ SME
3,VPTD,THANG,H321,3518,Doanh thu dịch vụ di động
4,VPTD,THANG,H321,3519,Doanh thu dịch vụ nhóm VTS


In [4]:
getListChart.head()

,SERVICEID,AREACODE,CYCLETIME,TYPECHART,CHARTID,TITLECHART,TYPE_CHART
0,157,A076,NAM,Biểu đồ đường,8340,AGG_DTDV CỐ ĐỊNH_TỪNG NĂM,LINE
1,20212510,A076,NAM,Biểu đồ đường,8368,AGG_DTDV NHÓM VDS_TỪNG NĂM,LINE
2,20214132,A076,NAM,Biểu đồ đường,8357,AGG_DTDV NHÓM VTS_TỪNG NĂM,LINE
3,20214133,A076,NAM,Biểu đồ đường,8326,AGG_DTDV NHÓM VTT_TỪNG NĂM,LINE
4,5066,A076,NAM,Biểu đồ đường,8348,AGG_DTDV SME_TỪNG NĂM,LINE


# TYPE

In [5]:
typeOutput2Input = {
    "Biểu đồ ghép cột": [
        "biểu đồ ghép cột",
        "ghép cột",
        "ghép cột chart",
        "chart ghép cột",
        "biểu đồ ghép column",
        "ghép column",
        "ghép column chart",
        "chart ghép column",
    ],
    "Biểu đồ table 2": [
        "biểu đồ table 2",
        "table 2",
        "table 2 chart",
        "chart table 2",
        "biểu đồ bảng 2",
        "bảng 2",
        "bảng 2 chart",
        "chart bảng 2",
    ],
    "Biểu đồ đường": [
        "Biểu đồ đường",
        "đường",
        "đường chart",
        "chart đường",
        "Biểu đồ line",
        "line",
        "line chart",
        "chart line",
    ],
    "Biểu đồ NESTED_PIE": [
        "Biểu đồ NESTED_PIE",
        "NESTED_PIE",
        "NESTED_PIE chart",
        "chart NESTED_PIE",
        "Biểu đồ NESTED PIE",
        "NESTED PIE",
        "NESTED PIE chart",
        "chart NESTED PIE",
        "Biểu đồ tròn NESTED",
        "tròn NESTED",
        "tròn NESTED chart",
        "chart tròn NESTED",
    ],
    "Biểu đồ INFO_OBJ": [
        "Biểu đồ INFO_OBJ",
        "INFO_OBJ",
        "INFO_OBJ chart",
        "chart INFO_OBJ",
        "Biểu đồ INFO OBJ",
        "INFO OBJ",
        "INFO OBJ chart",
        "chart INFO OBJ",
        "INFORMATION OBJECT",
    ],
    "Biểu đồ miền": ["Biểu đồ miền", "miền", "miền chart", "chart miền", "area chart"],
    "Biểu đồ cảnh báo 2": [
        "Biểu đồ cảnh báo 2",
        "cảnh báo 2",
        "cảnh báo 2 chart",
        "chart cảnh báo 2",
    ],
    "Biểu đồ cơ cấu": ["Biểu đồ cơ cấu", "cơ cấu", "cơ cấu chart", "chart cơ cấu"],
    "Biểu đồ BLOCK_INFO": [
        "Biểu đồ BLOCK_INFO",
        "BLOCK_INFO",
        "BLOCK_INFO chart",
        "chart BLOCK_INFO",
        "Biểu đồ BLOCK INFO",
        "BLOCK INFO",
        "BLOCK INFO chart",
        "chart BLOCK INFO",
    ],
    "Biểu đồ cơ cấu cột chồng": [
        "Biểu đồ cơ cấu cột chồng",
        "cơ cấu cột chồng",
        "cơ cấu cột chồng chart",
        "chart cơ cấu cột chồng",
        "Biểu đồ cơ cấu column chồng",
        "cơ cấu column chồng",
        "cơ cấu column chồng chart",
        "chart cơ cấu column chồng",
    ],
    "Biểu đồ cột": [
        "Biểu đồ cột",
        "cột",
        "cột chart",
        "chart cột",
        "Biểu đồ column",
        "column",
        "column chart",
        "chart column",
    ],
    "Biểu đồ cảnh báo": [
        "Biểu đồ cảnh báo",
        "cảnh báo",
        "cảnh báo chart",
        "chart cảnh báo",
    ],
    "Biểu đồ radar": ["Biểu đồ radar", "radar", "radar chart", "chart radar"],
    "Biểu đồ RANK": ["Biểu đồ RANK", "RANK", "RANK chart", "chart RANK"],
    "Biểu đồ ranking": ["Biểu đồ ranking", "ranking", "ranking chart", "chart ranking"],
    "Biểu đồ nhóm cột": [
        "Biểu đồ nhóm cột",
        "nhóm cột",
        "nhóm cột chart",
        "chart nhóm cột",
        "Biểu đồ column group",
        "column group",
        "column group chart",
        "column group",
    ],
    "Biểu đồ BLOCK_INFO_2": [
        "Biểu đồ BLOCK_INFO_2",
        "BLOCK_INFO_2",
        "BLOCK_INFO_2 chart",
        "chart BLOCK_INFO_2",
        "Biểu đồ BLOCK INFO 2",
        "BLOCK INFO 2",
        "BLOCK INFO 2 chart",
        "chart BLOCK INFO 2",
    ],
    "Biểu đồ TABLE": [
        "Biểu đồ TABLE",
        "TABLE",
        "TABLE chart",
        "chart TABLE",
        "Biểu đồ bảng",
        "bảng",
        "bảng chart",
        "chart bảng",
    ],
    "Biểu đồ INFO_MAIN": [
        "Biểu đồ INFO_MAIN",
        "INFO_MAIN",
        "INFO_MAIN chart",
        "chart INFO_MAIN",
        "Biểu đồ INFO MAIN",
        "INFO MAIN",
        "INFO MAIN chart",
        "chart INFO MAIN",
    ],
    "Biểu đồ kết hợp đường và cột": [
        "Biểu đồ kết hợp đường và cột",
        "kết hợp cột và đường",
        "kết hợp đường và cột chart",
        "chart kết hợp cột và đường",
        "Biểu đồ kết hợp column và line",
        "kết hợp line và column",
        "kết hợp column và line chart",
        "chart kết hợp line và column",
    ],
    "Biểu đồ tổng quan": [
        "Biểu đồ tổng quan",
        "tổng quan",
        "tổng quan chart",
        "chart tổng quan",
        "Biểu đồ overview",
        "overview",
        "overview chart",
        "chart overview",
    ],
    "Biểu đồ tương quan": [
        "Biểu đồ tương quan",
        "tương quan",
        "tương quan chart",
        "chart tương quan",
        "Biểu đồ correlation",
        "correlation",
        "correlation chart",
        "chart correlation",
    ],
    "Biểu đồ chữ T": [
        "Biểu đồ chữ T",
        "chữ T",
        "chữ T chart",
        "chart chữ T",
        "Biểu đồ word T",
        "char T",
        "char T chart",
        "chart word T",
    ],
    "Biểu đồ thị phần": [
        "Biểu đồ thị phần",
        "thị phần",
        "thị phần chart",
        "chart thị phần",
        "Biểu đồ pie",
        "pie",
        "pie chart",
        "chart pie",
    ],
    "Chart báo cáo": [
        "Chart báo cáo",
        "Biểu đồ báo cáo",
        "Chart report",
        "Biểu đồ report",
        "report",
        "báo cáo chart",
        "báo cáo biểu đồ",
        "report chart",
        "report biểu đồ",
    ],
    "Biểu đồ cột chồng": [
        "Biểu đồ cột chồng",
        "cột chồng",
        "cột chồng chart",
        "chart cột chồng",
        "Biểu đồ stacked column",
        "stacked column",
        "stacked column chart",
        "stacked column chart",
    ],
    "Bản Đồ": ["Bản Đồ", "map"],
    "LINE_DASHED chart": [
        "LINE_DASHED chart",
        "biểu đồ LINE_DASHED",
        "LINE_DASHED",
        "LINE DASHED chart",
        "biểu đồ LINE DASHED",
        "LINE DASHED",
        "đường DASHED chart",
        "biểu đồ đường DASHED",
        "đường DASHED",
    ],
    "Biểu đồ cột ngang": [
        "Biểu đồ cột ngang",
        "cột ngang",
        "cột ngang chart",
        "chart cột ngang",
        "Biểu đồ horizontal bar",
        "horizontal bar",
        "horizontal bar chart",
        "chart horizontal bar",
    ],
    "Dashboard": [
        "đát bót",
        "đát bót chỉ huy",
        "đát bọt",
        "đát bọt chỉ huy",
        "dashboard",
        "dashboard chỉ huy",
        "db",
        "db chỉ huy",
    ],
    "Khác": ["", "", "", "", "", "", "", ""],
}

In [6]:
typeOutput = list(typeOutput2Input.keys())
typeOutput[:5]

['Biểu đồ ghép cột',
 'Biểu đồ table 2',
 'Biểu đồ đường',
 'Biểu đồ NESTED_PIE',
 'Biểu đồ INFO_OBJ']

# Donvi

In [7]:
areacode2groupprofilecode = {}
for i in range(len(getDashboard)):
    if getDashboard['AREACODE'].loc[i] not in areacode2groupprofilecode.keys():
        areacode2groupprofilecode[getDashboard['AREACODE'].loc[i]]=[getDashboard['GROUPPROFILECODE'].loc[i]]
    elif getDashboard['GROUPPROFILECODE'].loc[i] not in areacode2groupprofilecode[getDashboard['AREACODE'].loc[i]]:
        areacode2groupprofilecode[getDashboard['AREACODE'].loc[i]].append(getDashboard['GROUPPROFILECODE'].loc[i])

areacode2groupprofilecode

{'H031': ['VPTD'],
 'H321': ['VPTD'],
 'VTZ': ['VPTD', 'BCL'],
 'MOV': ['VPTD', 'BCL'],
 'BC': ['VPTD', 'BCL', 'CSKH', 'CQCT', 'Dashboard Năm'],
 'ANM': ['VPTD', 'CQCT', 'BCL'],
 'VTS': ['VPTD',
  'BCL',
  'CSKH',
  'QTDL',
  'CNTT',
  'CQCT',
  'BAN_DO',
  'Dashboard Năm'],
 'T037': ['VPTD'],
 'T054': ['VPTD'],
 'H019': ['VPTD'],
 'N351': ['VPTD'],
 'H039': ['VPTD'],
 'B240': ['VPTD'],
 'VDTC': ['VPTD', 'CQCT'],
 'VTP': ['VPTD', 'BCL'],
 'XNK': ['VPTD', 'CQCT', 'BCL', 'DT_TC'],
 'XMCP': ['VPTD', 'CQCT', 'BCL'],
 'IDC': ['VPTD', 'CQCT', 'BCL', 'DT_TC'],
 'A076': ['VPTD'],
 'H711': ['VPTD'],
 'D067': ['VPTD'],
 'P057': ['VPTD'],
 'G059': ['VPTD'],
 'H320': ['VPTD'],
 'D061': ['VPTD'],
 'VBD': ['VPTD', 'BCL'],
 'VHK': ['VPTD', 'BCL'],
 'CTM': ['BCL', 'QTDL', 'VPTD'],
 'NAT': ['BCL', 'VPTD'],
 'VDS': ['BCL',
  'VPTD',
  'CSKH',
  'CNTT',
  'QTDL',
  'CQCT',
  'BAN_DO',
  'Dashboard tuần chỉ huy'],
 'VTL': ['BCL', 'VPTD'],
 'CT': ['VPTD', 'CQCT', 'BCL'],
 'TVTK': ['VPTD', 'CQCT', 'BCL'],
 

In [8]:
with open("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/demo/areacode2allgroupprofilecode.json", 'w') as f:
    json.dump(areacode2groupprofilecode, f, indent=4)

In [9]:
areacodeDb = getDashboard['AREACODE'].dropna()
print("Number of areacode in Dashboard API:", len(areacodeDb.unique()))
areacodeDb.unique()

Number of areacode in Dashboard API: 107


array(['H031', 'H321', 'VTZ', 'MOV', 'BC', 'ANM', 'VTS', 'T037', 'T054',
       'H019', 'N351', 'H039', 'B240', 'VDTC', 'VTP', 'XNK', 'XMCP',
       'IDC', 'A076', 'H711', 'D067', 'P057', 'G059', 'H320', 'D061',
       'VBD', 'VHK', 'CTM', 'NAT', 'VDS', 'VTL', 'CT', 'TVTK', 'TH', 'M3',
       'M1', 'VNC', 'VTTNEWNEW', 'VTC', 'STL', 'MYN', 'VTG', 'KGM',
       'Q053', 'Q510', 'D511', 'VTE', 'TD', 'TN', 'VTN', 'TTTT', 'B241',
       'T008', 'V064', 'V211', 'B650', 'T074', 'B781', 'T027', 'T280',
       'T073', 'T036', 'S079', 'S022', 'C026', 'Q055', 'C710', 'Q033',
       'D230', 'Q052', 'N068', 'N350', 'N030', 'L025', 'L063', 'H018',
       'L072', 'K060', 'K077', 'Y029', 'V070', 'T066', 'B651', 'B075',
       'C780', 'N038', 'K058', 'D500', 'B056', 'B281', 'SXTB', 'VTA',
       'HVT', 'BDS', 'D501', 'L020', 'H004', 'P210', 'TSS', 'B062',
       'L231', 'VMC', 'CNTT', 'KTM', 'DVK', 'KXL', 'H004003'],
      dtype=object)

In [42]:
areacode2areacodeText = {}
for areacode in areacodeDb.unique():
    db_name_list = getDashboard[getDashboard['AREACODE']==areacode]['DASHBOARDNAME'].unique()
    shortest_db_name = min(db_name_list, key=len)
    areacode2areacodeText[areacode] = shortest_db_name

areacode2areacodeText['T054'] = 'Thừa Thiên Huế'
areacode2areacodeText['VBD'] = 'VBD'
areacode2areacodeText['TVTK'] = 'TVTK'
areacode2areacodeText['TD'] = 'Tập đoàn'
areacode2areacodeText['V064'] = 'Bà Rịa Vũng Tàu'

{'L231': 'Lai Châu',
 'VMC': 'VMC',
 'CNTT': 'KHỐI CNTT',
 'KTM': 'KHỐI THƯƠNG MẠI VÀ LOGISTICS',
 'DVK': 'ĐƠN VỊ KHÁC',
 'KXL': 'KHỐI XÂY LẮP',
 'H004003': 'Q.Ba Đình'}

In [46]:
"""with open('/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/demo/data/areacode2areacodeText.json', 'w') as f:
    json.dump(areacode2areacodeText, f, indent=4)"""

In [41]:
getDashboard[getDashboard['AREACODE']=='TSS']['DASHBOARDNAME'].unique()

array(['TIÊU DÙNG THỰC DI ĐỘNG', 'Tiêu dùng thực di động', 'Khối tỉnh/TP',
       'Doanh thu dịch vụ (VTT+VTS+VDS)', 'Doanh thu dịch vụ nhóm VTS',
       'Thuê bao FTTH', 'Thuê bao di động phát triển thực',
       'Thuê bao 4G phát triển thực', 'Thuê bao register',
       'Doanh thu dịch vụ nhóm VTT', 'Doanh thu dịch vụ nhóm VDS'],
      dtype=object)

In [12]:
getDashboard[getDashboard['AREACODE']=='H321']

,GROUPPROFILECODE,CYCLETIME,AREACODE,DASHBOARDID,DASHBOARDNAME
2,VPTD,THANG,H321,3517,Doanh thu dịch vụ SME
3,VPTD,THANG,H321,3518,Doanh thu dịch vụ di động
4,VPTD,THANG,H321,3519,Doanh thu dịch vụ nhóm VTS
5,VPTD,THANG,H321,3520,Thuê bao FTTH
6,VPTD,THANG,H321,3521,Thuê bao di động phát triển thực
7,VPTD,THANG,H321,3522,Thuê bao 4G phát triển thực
8,VPTD,THANG,H321,3523,Tổng doanh thu dịch vụ
9,VPTD,THANG,H321,3524,Thuê bao register
775,VPTD,THANG,H321,6576,Hưng Yên
1444,VPTD,NGAY,H321,6400,Hưng Yên


In [270]:
def process_dv(dv):
    return dv.replace("'", "").replace('"', "").upper().strip()

areacodeChart = getListChart['AREACODE'].apply(process_dv).dropna()
print("Number of areacode in Chart API:", len(areacodeChart.unique()))
areacodeChart.unique()

Number of areacode in Dashboard API: 152


array(['A076', 'B650', 'B056', 'BDS', 'B240', 'VTP', 'B281', 'B781',
       'B241', 'B651', 'B075', 'B062', 'C026', 'C780', 'CTM', 'C710',
       'D230', 'D501', 'BQTDL', 'G059', 'D500', 'D511', 'D061', 'TH',
       'ANM', 'BC', 'CT', 'IDC', 'KGM', 'M1', 'M3', 'MOV', 'MYN', 'NAT',
       'STL', 'TD', 'TVTK', 'VBD', 'VDS', 'VDTC', 'VHK', 'VNC', 'VTC',
       'VTG', 'VTL', 'VTN', 'VTS', 'VTTNEWNEW', 'VTZ', 'XMCP', 'XNK',
       'VTT', 'TN', 'D067', 'CNTT', 'DVK', 'KTM', 'KXL', 'SXTB', 'MVT',
       'H018', 'T008', 'H320', 'H019', 'H004', 'N351', 'H031', 'H039',
       'T054', 'H711', 'H321', 'K077', 'K058', 'K060', 'L072', 'L020',
       'L231', 'L063', 'L025', 'N038', 'N030', 'N350', 'N068', 'P210',
       'P057', 'Q052', 'Q033', 'Q055', 'Q510', 'Q053', 'S022', 'S079',
       'SXTBCNC', 'VCR', 'T027', 'T036', 'T037', 'T066', 'T073', 'T074',
       'T280', 'V064', 'V070', 'V211', 'VCM', 'Y029', 'VMC', 'VTA', 'HVT',
       'VTE', 'TTTT', 'VDB', 'TV360', 'VTCM', 'VTTNEW', 'BCL', 'BCNTT',
 

In [271]:
areacodeOutput = list(set(areacodeDb.unique().tolist() + areacodeChart.unique().tolist()))
print("Number of areacode in 2 API:", len(areacodeOutput))
list(set(areacodeOutput))
for i in areacodeOutput:
    if i not in areacodeChart.unique():
        print(i)

Number of areacode in 2 API: 153
H004003


In [12]:
# Dashboard param tree
dashboard_param = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/DASHBOARD_PARAM_TREE_202307280939.csv")
dashboard_param.drop(columns=["PARAM_ORDER_SMS", "DEPT_ORDER", "SUB_CODE", "IMAGE",
                              "STATUS", "TYPE_PARAM", "GROUP_TYPE",
                              "START_TIME", "END_TIME",
                              "MODIFIED_DATE"], inplace=True)
dashboard_param.head()

,CODE,NAME,PARENT,PARAM_ORDER,LEVEL_NODE,ID,DEPT_ID,PARENT_DEPT_ID,STANDARD_NAME
0,VTG_METFONE,"Metfone_CT Viettel Cambodia Pte., Ltd",VTG,0,11.0,100676,308469,256.0,NaN
1,TTTT,VTSport_TT Thể thao Viettel,CPLTM,5,NaN,100698,2108,308480.0,NaN
2,HVT,VTAca_Học viện Viettel,CPLTM,4,NaN,100699,305965,308480.0,NaN
3,VTH,TT KD Sản phẩm công nghệ cao Viettel,SXTBCNC_HHL,1,NaN,100724,7920,308482.0,NaN
4,VTEK,TT Nghiên cứu công nghệ mạng Viettel,SXTBCNC_HHL,0,NaN,100725,1709,308482.0,NaN


In [13]:
dashboard_param = dashboard_param[['CODE', 'NAME', 'PARENT', 'STANDARD_NAME']]
dashboard_param.head()

,CODE,NAME,PARENT,STANDARD_NAME
0,VTG_METFONE,"Metfone_CT Viettel Cambodia Pte., Ltd",VTG,NaN
1,TTTT,VTSport_TT Thể thao Viettel,CPLTM,NaN
2,HVT,VTAca_Học viện Viettel,CPLTM,NaN
3,VTH,TT KD Sản phẩm công nghệ cao Viettel,SXTBCNC_HHL,NaN
4,VTEK,TT Nghiên cứu công nghệ mạng Viettel,SXTBCNC_HHL,NaN


In [14]:
count = 0
list_code = []
list_not_code = []
for area in areacodeOutput:
    if area in list(set(dashboard_param['CODE'].unique())):
        list_code.append(area)
        count += 1
    else:
        list_not_code.append(area)

print(count)
print(list_code)
print(list_not_code)

107
['N351', 'VTS', 'XMCP', 'BC', 'P210', 'VTZ', 'VDTC', 'VMC', 'VDS', 'SXTBCNC', 'M3', 'VNC', 'L025', 'T027', 'NAT', 'Q052', 'B781', 'TVTK', 'VTP', 'VBD', 'CPLTM', 'D500', 'VTR', 'Q053', 'D067', 'TH', 'M1', 'C026', 'MOV', 'H711', 'V064', 'D230', 'VTCM', 'STL', 'TD', 'H019', 'VTL', 'T008', 'Q033', 'B062', 'H320', 'P057', 'T280', 'T036', 'ANM', 'T073', 'H004003', 'D511', 'VTN', 'VTG', 'S022', 'C710', 'H018', 'VTE', 'T037', 'K077', 'L231', 'H039', 'H004', 'G059', 'VTC', 'L020', 'B075', 'B240', 'H031', 'T066', 'N038', 'B241', 'B281', 'L063', 'N350', 'K060', 'TSS', 'V070', 'V211', 'VT', 'XNK', 'N068', 'Y029', 'L072', 'CT', 'K058', 'D061', 'TTTT', 'IDC', 'VTTNEWNEW', 'MYN', 'BDS', 'S079', 'VHK', 'HVT', 'C780', 'KGM', 'N030', 'T074', 'T054', 'B056', 'Q055', 'VTA', 'H321', 'B650', 'B651', 'Q510', 'D501', 'A076', 'CTM', 'GPCNDV']
['DVK', 'TT', 'VCM', 'VPTD', 'TC', 'VDB', 'VTNN', 'TN', 'BTT', 'MVT', 'KTM', 'XNKVG', 'BTTKT', 'BQTDL', 'VCR', 'VP', 'QLDA', 'CNTT', 'T079', 'BCNC', 'BHC', 'BDVDC', '

In [16]:
with open("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data_creation/shortestdonvi2all.json", 'r') as f:
    shortestdonvi2all = json.load(f)

shortestdonvi2all = {u.upper():v for u, v in shortestdonvi2all.items()}

In [18]:
areacodeOutput2Input = {}
areacodeRemain = deepcopy(areacodeOutput)
for areacode in areacodeOutput:
    if areacode in shortestdonvi2all.keys():
        areacodeRemain.remove(areacode)
        areacodeOutput2Input[areacode] = shortestdonvi2all[areacode]
        continue

    if areacode in list_not_code:
        areacodeRemain.remove(areacode)
        areacodeOutput2Input[areacode] = [areacode]
        continue

    if areacode in list_code:
        inst = dashboard_param[dashboard_param['CODE']==areacode]
        areacodeRemain.remove(areacode)
        areacodeOutput2Input[areacode] = []
        if isinstance(inst['STANDARD_NAME'].tolist()[0], str):
            if "Q." in inst['STANDARD_NAME'].tolist()[0]:
                areacodeOutput2Input[areacode].append(inst['STANDARD_NAME'].tolist()[0].replace("Q.", "Quận "))
            else:
                areacodeOutput2Input[areacode].append(inst['STANDARD_NAME'].tolist()[0])

        if '_' in inst['NAME'].tolist()[0]:
            areacodeOutput2Input[areacode].append(inst['NAME'].tolist()[0].split("_")[-1])
        else:
            areacodeOutput2Input[areacode].append(inst['NAME'].tolist()[0])
        continue

for areacode in areacodeOutput:
    areacodeOutput2Input[areacode].append(areacode)
    areacodeOutput2Input[areacode] = list(set(areacodeOutput2Input[areacode]))
    check = deepcopy(areacodeOutput2Input[areacode])
    for inputt in check:
        if inputt in shortestdonvi2all.keys():
            areacodeOutput2Input[areacode] += shortestdonvi2all[inputt]
    areacodeOutput2Input[areacode] = [i.lower() for i in areacodeOutput2Input[areacode]]
    areacodeOutput2Input[areacode] = list(set(areacodeOutput2Input[areacode]))

my_added = {
    'CNTT': ['công nghệ thông tin'],
    'BCNTT': ['ban cntt', 'ban công nghệ thông tin'],
    'BCL': ['ban chiến lược', 'ban cl'],
    'VT': ['viễn thông'],
    'BCNC': ['ban công nghệ cao'],
    'VPTD': ['văn phòng tập đoàn', 'vp tập đoàn', 'văn phòng tđ'], 
}

for areacode in my_added.keys():
    areacodeOutput2Input[areacode].append(my_added[areacode])

# SERVICEID + DASHBOARD_NAME

## getDashboard

In [19]:
getDashboard

,GROUPPROFILECODE,CYCLETIME,AREACODE,DASHBOARDID,DASHBOARDNAME
0,VPTD,NGAY,H031,3581,Doanh thu dịch vụ nhóm VTT
1,VPTD,NGAY,H031,3582,Doanh thu dịch vụ nhóm VDS
2,VPTD,THANG,H321,3517,Doanh thu dịch vụ SME
3,VPTD,THANG,H321,3518,Doanh thu dịch vụ di động
4,VPTD,THANG,H321,3519,Doanh thu dịch vụ nhóm VTS
...,...,...,...,...,...
3935,DB_CNTT,THANG,VTTNEWNEW,11159,HỆ THỐNG ĐƯỢC TÍCH HỢP VÀO DOC
3936,DB_CNTT,THANG,VTTNEWNEW,11160,LEAD TIME
3937,DT_TC,THANG,XNK,11216,IDC_11216
3938,DT_TC,THANG,XNK,11217,LỢI NHUẬN TRƯỚC THUẾ_11217


In [313]:
configDashboard = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/CONFIG_DASHBOARD_202309051338.csv")
configDashboard.drop(columns=['UPDATE_TIME', 'UPDATE_USER', 'DESCRIPTION', 'STATUS'], inplace=True)
dashboard_name_donvi = configDashboard[configDashboard['DASHBOARD_TYPE']==1]['DASHBOARD_NAME'].unique()
dashboard_name_in_donvi = ['TƯ VẤN THIẾT KẾ', 'CÔNG TRÌNH', 'MEDIA', 'Kết quả sản xuất kinh doanh', 'Tổng doanh thu']
dashboard_name_donvi = [i for i in dashboard_name_donvi if i not in dashboard_name_in_donvi]
dashboard_name_donvi+=['MAP', 'MOCHA', 'VIETTELPAY', 'MYVIETTEL', 'TV360', 'VOFFICE', 'DMS MỘC CHÂU', 'IM2', 'VIETTEL', 'SALE', 'MBCCS', 'K12ONLINE', 'PAYMENT', 'Mặc định', 'VTNN']
getDashboardName = getDashboard[~getDashboard['DASHBOARDNAME'].isin(dashboard_name_donvi)]
getDashboardName['check_'] = getDashboardName['DASHBOARDNAME'].apply(lambda x: 1 if x.find("_")>-1 else 0)
getDashboardName['DASHBOARD_NAME_NEW'] = getDashboardName['DASHBOARDNAME'].str.strip()
for _ in range(3):
    getDashboardName['check_1'] = getDashboardName['DASHBOARD_NAME_NEW'].apply(lambda x: 1 if x.find("_")>-1 else 0)
    getDashboardName['check_2'] = getDashboardName.apply(lambda x: 1 if (x['check_1']==1 and (
        x['DASHBOARD_NAME_NEW'][x['DASHBOARD_NAME_NEW'].rfind("_")+1:].strip().isnumeric() 
        or x['DASHBOARD_NAME_NEW'][x['DASHBOARD_NAME_NEW'].rfind("_")+1:].strip()=='Tháng'
        or x['DASHBOARD_NAME_NEW'][x['DASHBOARD_NAME_NEW'].rfind("_")+1:].strip() in dashboard_name_donvi
    )) else 0, axis=1)
    getDashboardName['DASHBOARD_NAME_NEW'] = getDashboardName.apply(lambda x: x['DASHBOARD_NAME_NEW'][:x['DASHBOARD_NAME_NEW'].find("_")].strip() if x['check_2']==1 else x['DASHBOARD_NAME_NEW'], axis=1)

getDashboardName['check_5'] = getDashboardName.apply(lambda x: 1 if (
    x['check_']==1 and 
    x['DASHBOARD_NAME_NEW'][:x['DASHBOARD_NAME_NEW'].find("_")] in dashboard_name_donvi
) else 0, axis=1)
getDashboardName['DASHBOARD_NAME_NEW'] = getDashboardName.apply(lambda x: x['DASHBOARD_NAME_NEW'][x['DASHBOARD_NAME_NEW'].find("_")+1:].strip() if x['check_5']==1 else x['DASHBOARD_NAME_NEW'], axis=1)
getDashboardName['check_6'] = getDashboardName.apply(lambda x: 1 if (
    x['check_1']==1 and 
    x['DASHBOARD_NAME_NEW'][:x['DASHBOARD_NAME_NEW'].find("_")] in dashboard_name_donvi
) else 0, axis=1)
getDashboardName['DASHBOARD_NAME_NEW'] = getDashboardName.apply(lambda x: x['DASHBOARD_NAME_NEW'][x['DASHBOARD_NAME_NEW'].find("_")+1:].strip() if x['check_6']==1 else x['DASHBOARD_NAME_NEW'], axis=1)

getDashboardName['check_3'] = getDashboardName['DASHBOARD_NAME_NEW'].apply(lambda x: 1 if x.find(".")>-1 else 0)
getDashboardName['check_4'] = getDashboardName.apply(lambda x: 1 if (x['check_3']==1 and x['DASHBOARD_NAME_NEW'][:x['DASHBOARD_NAME_NEW'].find(".")].strip().isnumeric()) else 0, axis=1)
getDashboardName['DASHBOARD_NAME_NEW'] = getDashboardName.apply(lambda x: x['DASHBOARD_NAME_NEW'][x['DASHBOARD_NAME_NEW'].find(".")+1:].strip() if x['check_4']==1 else x['DASHBOARD_NAME_NEW'], axis=1)
getDashboardName = getDashboardName[~getDashboardName['DASHBOARD_NAME_NEW'].str.contains('Copy')]
getDashboardName = getDashboardName[~getDashboardName['DASHBOARD_NAME_NEW'].str.contains('dashboard')]
getDashboardName['check_6'] = getDashboardName['DASHBOARD_NAME_NEW'].apply(lambda x: 1 if (
    x.find("_")>-1
    and len(x.split("_"))==2
    and x.split("_")[0].isnumeric()
    and x.split("_")[1].isnumeric()
) else 0)
getDashboardName = getDashboardName[getDashboardName['check_6']==0]
getDashboardName = getDashboardName[~getDashboardName['DASHBOARD_NAME_NEW'].isin(['DGD', 'MEDIA', 'IDC'])]
chitieuDashboardOutput = list(getDashboardName['DASHBOARD_NAME_NEW'].str.lower().unique())
print(len(chitieuDashboardOutput))
chitieuDashboardOutput[-5:]

437


['kpi hồ sơ quyết toán công trình',
 'kpi nghiệm thu đề tài khcn',
 'số lượng bài viết phản ánh',
 'số lượng bài viết truyền thông',
 'theo dõi số lượng bài viết cb cnv']

## getListChart


In [314]:
getListChart

,SERVICEID,AREACODE,CYCLETIME,TYPECHART,CHARTID,TITLECHART,TYPE_CHART
0,157,A076,NAM,Biểu đồ đường,8340,AGG_DTDV CỐ ĐỊNH_TỪNG NĂM,LINE
1,20212510,A076,NAM,Biểu đồ đường,8368,AGG_DTDV NHÓM VDS_TỪNG NĂM,LINE
2,20214132,A076,NAM,Biểu đồ đường,8357,AGG_DTDV NHÓM VTS_TỪNG NĂM,LINE
3,20214133,A076,NAM,Biểu đồ đường,8326,AGG_DTDV NHÓM VTT_TỪNG NĂM,LINE
4,5066,A076,NAM,Biểu đồ đường,8348,AGG_DTDV SME_TỪNG NĂM,LINE
...,...,...,...,...,...,...,...
35508,1314,STL,THANG,Biểu đồ cột,5332,rpm thoại Xu hướng tháng,COLUMN
35509,1144,TD,THANG,Biểu đồ cột chồng,21918,test cột chồng,STACK
35510,143,TD,THANG,Biểu đồ cột chồng,21918,test cột chồng,STACK
35511,1202103,VDS,THANG,Biểu đồ đường,22573,ĐIỂM CHẤP NHẬN THANH TOÁN PSGD_TỪNG THÁNG,LINE


In [529]:
service_id_all = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/SERVICE_GBTD_202309051334.csv")
service_id_list = list(set(service_id_all['SERVICE_ID']))
service_id_dictionary = {u: list(set(service_id_all[service_id_all['SERVICE_ID']==u]['SERVICE_NAME']))[0] for u in service_id_list}

In [603]:
service_id_dictionary

{1.0: 'test',
 2.0: 'Doanh thu dịch vụ',
 3.0: 'TD THỰC CÁC DV (TS+TT)',
 4.0: 'TD THỰC DV DI ĐỘNG (TS+TT)',
 5.0: 'TD GỐC DI ĐỘNG TT',
 6.0: 'TB REGISTER',
 7.0: 'Thuê bao 3K3D/15D3D di động (TT+TS)',
 8.0: 'TB 20K/1$ DI ĐỘNG TT',
 9.0: 'Thuê bao 4G tăng giảm',
 12.0: 'Doanh thu Viettel High Tech',
 14.0: 'chỉ tiêu 13',
 17.0: 'chỉ tiêu không hoạt động',
 19.0: 'TỔNG DOANH THU (chi tiết)',
 20.0: 'Doanh thu di động',
 21.0: 'bán thẻ',
 22.0: 'thẻ cào giấy',
 23.0: 'pincode',
 24.0: 'Anypay',
 25.0: 'Topup',
 26.0: 'Thanh toán TT qua Bankplus',
 27.0: 'Nạp thẻ',
 28.0: 'Cho các DV TT',
 29.0: 'Cho các DV TS (-)',
 30.0: 'Cho bên thứ 3 (game,…)',
 31.0: 'Thiết bị',
 32.0: 'Điện thoại 2G',
 33.0: 'Hàng Viettel (O.E.M)',
 34.0: 'Hàng hãng',
 35.0: 'Điện thoại 3G',
 36.0: 'Featurephone',
 37.0: 'Hàng Viettel (O.E.M)',
 38.0: 'Hàng hãng',
 39.0: 'Smartphone',
 40.0: 'Hàng Viettel (O.E.M)',
 41.0: 'Hàng hãng',
 42.0: 'Thiết bị khác',
 44.0: 'USB',
 45.0: 'Tablet',
 46.0: 'Máy CĐKD (Hphone, 

In [606]:
chitieu2serviceid = {}
for id, chitieu in service_id_dictionary.items():
    if chitieu.lower() not in chitieu2serviceid.keys():
        chitieu2serviceid[chitieu.lower()] = int(id)

In [609]:
with open('/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/demo/chitieu2serviceid.json', 'w') as f:
    json.dump(chitieu2serviceid, f, indent=4)

In [607]:
chitieu2serviceid

{'test': 1,
 'doanh thu dịch vụ': 2,
 'td thực các dv (ts+tt)': 3,
 'td thực dv di động (ts+tt)': 4,
 'td gốc di động tt': 5,
 'tb register': 6,
 'thuê bao 3k3d/15d3d di động (tt+ts)': 7,
 'tb 20k/1$ di động tt': 8,
 'thuê bao 4g tăng giảm': 9,
 'doanh thu viettel high tech': 12,
 'chỉ tiêu 13': 14,
 'chỉ tiêu không hoạt động': 17,
 'tổng doanh thu (chi tiết)': 19,
 'doanh thu di động': 20,
 'bán thẻ': 21,
 'thẻ cào giấy': 22,
 'pincode': 23,
 'anypay': 24,
 'topup': 25,
 'thanh toán tt qua bankplus': 26,
 'nạp thẻ': 27,
 'cho các dv tt': 28,
 'cho các dv ts (-)': 29,
 'cho bên thứ 3 (game,…)': 30,
 'thiết bị': 31,
 'điện thoại 2g': 32,
 'hàng viettel (o.e.m)': 33,
 'hàng hãng': 34,
 'điện thoại 3g': 35,
 'featurephone': 36,
 'smartphone': 39,
 'thiết bị khác': 42,
 'usb': 44,
 'tablet': 45,
 'máy cđkd (hphone, cityphone': 46,
 'khác (router 3g,…)': 47,
 'doanh thu hòa mạng và dv sau bán': 48,
 'hòa mạng tt': 49,
 'hòa mạng ts': 50,
 'dv sau bán': 51,
 'cước phát sinh ts': 52,
 'di độ

In [530]:
getListChartName = deepcopy(getListChart)
getListChartName['SERVICE_NAME'] = getListChartName['SERVICEID'].apply(lambda i: service_id_dictionary[i] if i in service_id_dictionary.keys() else "")
getListChartName.head()

,SERVICEID,AREACODE,CYCLETIME,TYPECHART,CHARTID,TITLECHART,TYPE_CHART,SERVICE_NAME
0,157,A076,NAM,Biểu đồ đường,8340,AGG_DTDV CỐ ĐỊNH_TỪNG NĂM,LINE,Doanh thu cố định
1,20212510,A076,NAM,Biểu đồ đường,8368,AGG_DTDV NHÓM VDS_TỪNG NĂM,LINE,DT dịch vụ nhóm DV của VDS
2,20214132,A076,NAM,Biểu đồ đường,8357,AGG_DTDV NHÓM VTS_TỪNG NĂM,LINE,DT dịch vụ nhóm DV của VTS
3,20214133,A076,NAM,Biểu đồ đường,8326,AGG_DTDV NHÓM VTT_TỪNG NĂM,LINE,DT dịch vụ nhóm DV của VTT
4,5066,A076,NAM,Biểu đồ đường,8348,AGG_DTDV SME_TỪNG NĂM,LINE,Doanh thu dịch vụ SME


In [531]:
chitieuChartOutput = list(getListChartName['SERVICE_NAME'].unique())
chitieuChartOutput[:5]

['Doanh thu cố định',
 'DT dịch vụ nhóm DV của VDS',
 'DT dịch vụ nhóm DV của VTS',
 'DT dịch vụ nhóm DV của VTT',
 'Doanh thu dịch vụ SME']

## Chi tieu output

In [532]:
chitieuOutput = chitieuChartOutput + chitieuDashboardOutput
random.shuffle(chitieuOutput)
chitieuOutput[:5]

['tỷ lệ đoàn khách tiếp đón thành công',
 'Trạm Trảng Bom (Đồng Thuận)',
 'Trạm Đắk Song',
 'Hoạt động tài chính',
 'Dịch vụ OS']

## Chi tieu output2input

In [536]:
def remove_donvi(title_chart):
    features = title_chart.split("_")
    if " " not in features[0] and features[0] not in ["company", 'info', 'test', 'rank', 'rada', 'table', 'tooltip', '21', 'dgsdf', 'khdn', 'dt-cp-ln', 'arpu', 'mou', 'dou']:
        output =  features[0]
        output2 = "_".join(features[1:])
    elif features[0] in ['công ty mẹ', 'tập đoàn', 'ban cntt', 'khối tỉnh/tp', 'công đoàn', 'viettel tỉnh/tp', 'viettel tỉnh tp']:
        output = features[0]
        output2 = "_".join(features[1:])
    elif len(features) == 4 and " " not in features[2]:
        output = features[2]
        output2 = "_".join(features[:2]+features[3:])
    else:
        output = ""
        output2 = "_".join(features[0:])
    return output, output2
    
def remove_time(title_chart_rm_donvi):
    features = title_chart_rm_donvi.split("_")
    if len(features) == 1:
        features = "_".join(features)
        return features, ""
    output2 = ""
    output = deepcopy(features)
    for feature in features:
        for i in ['ngay', 'thang', 'nam', 'quy', 'ngày', 'tháng', 'năm', 'quý', 'tết', 'qúy']:
            if i in feature:
                output2 = feature
                output.remove(feature)
                break
    
    output = "_".join(output)
    return output, output2

def remove_redundant(title_chart_re_donvi_time):
    features = title_chart_re_donvi_time.split("_")
    output2 = ""
    output = deepcopy(features)
    if len(features) > 1:
        for feature in features:
            feature2 = feature.strip()
            if feature2 in ['line', 
                           'theo hướng', 
                           'xu huong', 
                           'theo cơ cấu', 
                           'cơ cấu từng khối', 
                           'cơ cấu cơ cấu từng khối', 
                           'cơ cấu dịch vụ', 
                           'cơ cấu đơn vị (lk kệ thống)',
                           'cơ cấu khối',
                           'các đơn vị',
                           'cơ cấu đơn vị',
                           'từng đơn vị',
                           'các đơn vị con nội bộ',
                           'cơ cấu %',
                           'theo loại thuê bao',
                           ] \
                or "theo hạ tầng" in feature:
                output2 = feature2
                output.remove(feature)
                break

            if 'test' in feature:
                output2 = feature
                output.remove(feature)
    output = "_".join(output)
    return output, output2

def check_final_name(row):
    if row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'] in row['TYPECHART']:
        return ""
    
    if row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'].isnumeric():
        return ""
    
    list_check2 = ['infomain', 'info_main', 'check', 'làm tròn', 'thoa', 'hà', 'copy', 'mai']
    for item in list_check2:
        if item in row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT']:
            return ""

    if row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'][0].isnumeric() \
        and row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'][1] in [' ', '.']:
        output = row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'][2:].strip()
    else:
        output = row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'].strip()

    return output

def check_final_name_for_input(output):
    if len(output.split("_")) > 1:
        output = " ".join([i.strip() for i in output.split("_")])
    return output

In [589]:
chart_dept_for_title = deepcopy(getListChartName)
chart_dept_for_title = chart_dept_for_title[['SERVICEID', 'AREACODE', 'CYCLETIME', 'TITLECHART', 'SERVICE_NAME', 'TYPECHART']]
chart_dept_for_title = chart_dept_for_title[~chart_dept_for_title['TITLECHART'].str.contains('test')]
chart_dept_for_title = chart_dept_for_title[~chart_dept_for_title['TITLECHART'].str.contains('copy')]
chart_dept_for_title = chart_dept_for_title.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
chart_dept_for_title['CHECK_LEN'] = chart_dept_for_title['TITLECHART'].apply(lambda x: len(x.split('_')))
chart_dept_for_title['DONVI_FEATURE'] = chart_dept_for_title['TITLECHART'].apply(lambda x: remove_donvi(x)[0])
chart_dept_for_title['TITLE_CHART_RM_DONVI'] = chart_dept_for_title['TITLECHART'].apply(lambda x: remove_donvi(x)[1])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[0])
chart_dept_for_title.drop(columns=['TITLE_CHART_RM_DONVI', 'CHECK_LEN'], inplace=True)
chart_dept_for_title['REDUNDANT'] = chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'].apply(lambda x: remove_redundant(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'] = chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'].apply(lambda x: remove_redundant(x)[0])
chart_dept_for_title['FINAL_TITLE'] = chart_dept_for_title.apply(lambda x: check_final_name(x), axis=1)
chart_dept_for_title['FINAL_TITLE_FOR_INPUT'] = chart_dept_for_title['FINAL_TITLE'].apply(check_final_name_for_input)
chart_dept_for_title = chart_dept_for_title[['TITLECHART', 'SERVICEID', 'SERVICE_NAME', 'FINAL_TITLE', 'FINAL_TITLE_FOR_INPUT']]
chart_dept_for_title.reset_index(inplace=True)
chart_dept_for_title.head()

,index,TITLECHART,SERVICEID,SERVICE_NAME,FINAL_TITLE,FINAL_TITLE_FOR_INPUT
0,0,agg_dtdv cố định_từng năm,157,doanh thu cố định,dtdv cố định,dtdv cố định
1,1,agg_dtdv nhóm vds_từng năm,20212510,dt dịch vụ nhóm dv của vds,dtdv nhóm vds,dtdv nhóm vds
2,2,agg_dtdv nhóm vts_từng năm,20214132,dt dịch vụ nhóm dv của vts,dtdv nhóm vts,dtdv nhóm vts
3,3,agg_dtdv nhóm vtt_từng năm,20214133,dt dịch vụ nhóm dv của vtt,dtdv nhóm vtt,dtdv nhóm vtt
4,4,agg_dtdv sme_từng năm,5066,doanh thu dịch vụ sme,dtdv sme,dtdv sme


In [596]:
chart_dept_for_title_2 = deepcopy(chart_dept_for_title)
ChitieuTieude2InputChart = {}
for i in range(len(chart_dept_for_title_2)):
    key = chart_dept_for_title_2.loc[i, 'SERVICE_NAME'] + "...." + chart_dept_for_title_2.loc[i, 'FINAL_TITLE'] 
    if key not in ChitieuTieude2InputChart.keys():
        ChitieuTieude2InputChart[key] = [chart_dept_for_title_2.loc[i, 'FINAL_TITLE_FOR_INPUT']]
        ChitieuTieude2InputChart[key].append(chart_dept_for_title_2.loc[i, 'SERVICE_NAME'])
    elif chart_dept_for_title_2.loc[i, 'FINAL_TITLE_FOR_INPUT'] not in ChitieuTieude2InputChart[key]:
        ChitieuTieude2InputChart[key].append(chart_dept_for_title_2.loc[i, 'FINAL_TITLE_FOR_INPUT'])

# check from 1 input to many output
InputChart2ChitieuTieude = {}
for ChitieuTieude in ChitieuTieude2InputChart.keys():
    for InputChart in ChitieuTieude2InputChart[ChitieuTieude]:
        if InputChart not in InputChart2ChitieuTieude.keys():
            InputChart2ChitieuTieude[InputChart] = [ChitieuTieude]
        else:
            InputChart2ChitieuTieude[InputChart].append(ChitieuTieude)

InputChart2ChitieuTieudeUnique = {u: InputChart2ChitieuTieude[u] for u in InputChart2ChitieuTieude.keys() if len(InputChart2ChitieuTieude[u]) < 2}

remainChiTieu = list(set([i[0][:i[0].find('....')] for i in InputChart2ChitieuTieudeUnique.values()]))
allChiTieu = chart_dept_for_title_2['SERVICE_NAME'].unique()
count = 0
for i in allChiTieu:
    if i not in remainChiTieu:
        InputChart2ChitieuTieudeUnique[i] = [i+"...."]

chitieuTieudeOutput2input = {InputChart2ChitieuTieudeUnique[i][0]: i for i in InputChart2ChitieuTieudeUnique}